In [164]:
import os
import json
import requests
import pandas as pd
import numpy as np
from lxml import html

In [165]:
def save_incremental():
    
    return 0

In [166]:
domain = 'realestate.co.jp'
url = 'https://realestate.co.jp/en/rent?prefecture=JP-13&city=13123&trainline=&district=&min_price=&max_price=&min_meter=&rooms=&distance_station=&agent_id=&building_type=&building_age=&updated_within=&transaction_type=&order=&search=Search&page='


In [167]:
html_content = requests.get(url)

In [168]:
tree = html.fromstring(html_content.content)

In [169]:
page_limit = tree.xpath('//ul[@class="paginator"]/li/text()')
page_limit = [d.strip() for d in page_limit]
page_limit = [d for d in page_limit if d.strip()]
page_limit = page_limit [:int(len(page_limit)/2)]
page_limit = ' '.join(page_limit)
page_limit[-3:]

'316'

In [170]:
body = tree.xpath('//div[@class="listing-body"]//div[@class="listing-item"]//text()')
body = [d.strip() for d in body]
body = [d for d in body if d.strip()]

In [171]:
new_list = []
for key, item in enumerate(body):
    if item == 'Available Now':
        new_list.append('Available From')
        new_list.append(item)
        continue
    
    if item == 'Year Built' and body[key-2] != 'Floor':
        new_list.append('Floor')
        new_list.append('0')
        new_list.append(item)
        continue
    
    new_list.append(item)
body = new_list
del new_list

In [172]:
pairs = list(zip(body[::2], body[1::2]))
n = 8
properties = [dict(pairs[i:i+n]) for i in range(0, len(pairs), n)]
print(json.dumps(properties, indent=2))

[
  {
    "Monthly Costs": "\u00a5150,000",
    "Available From": "Oct 15, 2025",
    "Size": "92.00 m\u00b2",
    "Deposit": "\u00a5140,000",
    "Key Money": "\u00a50",
    "Floor": "2 / 4F",
    "Year Built": "1994",
    "Nearest Station": "Funabori Station (24 min. walk)"
  },
  {
    "Monthly Costs": "\u00a5137,000",
    "Available From": "Oct 15, 2025",
    "Size": "55.71 m\u00b2",
    "Deposit": "\u00a5129,000",
    "Key Money": "\u00a50",
    "Floor": "4 / 9F",
    "Year Built": "1995",
    "Nearest Station": "Kasai Station (8 min. walk)"
  },
  {
    "Monthly Costs": "\u00a565,550",
    "Available From": "Oct 15, 2025",
    "Size": "19.87 m\u00b2",
    "Deposit": "\u00a50",
    "Key Money": "\u00a50",
    "Floor": "1 / 2F",
    "Year Built": "2005",
    "Nearest Station": "Koiwa Station (17 min. walk)"
  },
  {
    "Monthly Costs": "\u00a560,000",
    "Available From": "Oct 15, 2025",
    "Size": "17.00 m\u00b2",
    "Deposit": "\u00a50",
    "Key Money": "\u00a50",
    "Floor

In [173]:
links = tree.xpath('//div[@class="listing-body"]//a/@href')
new_list = []
for key, link in enumerate(links):
    if link != links[key-1]:
        new_list.append(link)
links = new_list
del new_list
links

['/en/rent/view/929228?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=0',
 '/en/rent/view/889867?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=1',
 '/en/rent/view/927321?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=2',
 '/en/rent/view/1206745?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=3',
 '/en/rent/view/1274217?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=4',
 '/en/rent/view/1272825?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=5',
 '/en/rent/view/1274920?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=6',
 '/en/rent/view/752790?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=7',
 '/en/rent/view/1273105?prefecture=JP-13&city=13123&search=Search&order=index_ranking-desc&page=1&index=8',
 '/en/rent/view/389472?prefectur

In [174]:
df = pd.DataFrame(properties)
df['Link'] = links
df

,Monthly Costs,Available From,Size,Deposit,Key Money,Floor,Year Built,Nearest Station,Link
0,"¥150,000","Oct 15, 2025",92.00 m²,"¥140,000",¥0,2 / 4F,1994,Funabori Station (24 min. walk),/en/rent/view/929228?prefecture=JP-13&city=131...
1,"¥137,000","Oct 15, 2025",55.71 m²,"¥129,000",¥0,4 / 9F,1995,Kasai Station (8 min. walk),/en/rent/view/889867?prefecture=JP-13&city=131...
2,"¥65,550","Oct 15, 2025",19.87 m²,¥0,¥0,1 / 2F,2005,Koiwa Station (17 min. walk),/en/rent/view/927321?prefecture=JP-13&city=131...
3,"¥60,000","Oct 15, 2025",17.00 m²,¥0,¥0,1 / 6F,1987,Funabori Station (5 min. walk),/en/rent/view/1206745?prefecture=JP-13&city=13...
4,"¥222,200",Available Now,64.37 m²,"¥200,000","¥200,000",8 / 9F,1990,Hirai Station (6 min. walk),/en/rent/view/1274217?prefecture=JP-13&city=13...
5,"¥234,000",Available Now,58.66 m²,¥0,"¥219,000",4 / 9F,2025,Nishikasai Station (9 min. walk),/en/rent/view/1272825?prefecture=JP-13&city=13...
6,"¥260,580","Oct 31, 2025",85.78 m²,"¥258,000","¥258,000",0,2023,Shinozaki Station (10 min. walk),/en/rent/view/1274920?prefecture=JP-13&city=13...
7,"¥45,000","Oct 15, 2025",10.37 m²,¥0,"¥42,000",1 / 2F,2014,Keiseikoiwa Station (3 min. walk),/en/rent/view/752790?prefecture=JP-13&city=131...
8,"¥85,000","Oct 10, 2025",35.21 m²,¥0,¥0,2 / 2F,2005,Ichinoe Station (8 min. walk),/en/rent/view/1273105?prefecture=JP-13&city=13...
9,"¥79,574",Available Now,22.35 m²,¥0,"¥73,000",1 / 2F,2008,"6 min. walk to 東小岩一丁目 bus stop, then 8 min. by...",/en/rent/view/389472?prefecture=JP-13&city=131...
